## Building Agents with RAG and Tools

### Setting up LLM & Loading the ENV file

In [ ]:
from dotenv import load_dotenv
from langchain_ollama import ChatOllama

load_dotenv(override=True)
load_dotenv('./../.env')

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="qwen3:latest",
    temperature=0.5,
    max_tokens=400
)

### Importing HuggingFace & Trasformers libraries

In [ ]:
# %pip install transformers
# %pip install langchain-huggingface
# %pip install torch torchvision
# %pip install -U sentence-transformers

### Setting up PlayWright

In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
import nest_asyncio

nest_asyncio.apply()

In [ ]:
async_chrome = create_async_playwright_browser()

toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_chrome)
list_of_tools = toolkit.get_tools()
count_of_tools = len(list_of_tools)
count_of_tools

### Setting up VectorDB & Retriever

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma(
    persist_directory='./chroma_db_hf', 
    embedding_function=embedding_model
)

result = vector_store.similarity_search("Which country is Parveen allowed to travel ?", k=5)

for doc in result:
    print(doc.metadata["source"])

### Setting up Retriever for the VectorDB

In [ ]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs = {'k': 5})

# response = retriever.invoke("Full name of Dhanesh?")

# for doc in response:
#     print(doc.metadata["source"])


### Creating a tool to Retrieve from the VectorDB

In [ ]:
from langchain.tools import tool

@tool
def retrieve_person_detail(query: str) -> str:
    """
    This tool provides employment pass details related to DHANESH and PARVEEN.
    You should use this tool to check any employment information related to Dhanesh and Parveen.
    
    Args:
        query: The search query
    """
    
    return retriever.invoke(query)

### Addition to the ToolKit

In [ ]:
if (count_of_tools < len(list_of_tools)):
    list_of_tools.remove(retrieve_person_detail)
    
if (count_of_tools == len(list_of_tools)):
    list_of_tools.append(retrieve_person_detail)
    
list_of_tools


### Creating Agent code to call the Tools

In [ ]:
from langchain.agents import AgentType, initialize_agent

agent = initialize_agent(
    tools=list_of_tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

query = "Following URL shows the Job information of Dhanesh - https://www.weekday.works/people/dhanesh-gujrathi-dhanesh-gujrathi-9129ab55. Can you check where was Dhanesh employed before Nityo ?"
# query = "Who applied the Employment pass for Dhanesh in 2019 ? Was it Nityo or NTT Data ?"
# query = "Who applied the Employment pass for Dhanesh in 2020 ? Was it Nityo or NTT Data ?"

response = await agent.ainvoke(query)

response